In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from feature_learning import *
from load_and_preprocessing import load_data_and_filter_members

C:\Users\silus\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\silus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
leg='50'
from_date = None
to_date = None
link_cutoff = 0.5
data_transf, adjacency, node_index, affairs_features, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-'+leg+'-0.csv',
                                                                     start_date=from_date, end_date=to_date,
                                                                     filter_method='number_NA',cutoff=10,ret_transf=True, delete_links_below=link_cutoff )

(Nbr. of councillors, nbr. of votes) before filter: (221, 1320)
(Nbr. of councillors, nbr. of votes) after filter: (175, 1320)


Put train data import here after

In [4]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')

X_train = X_train[y_train != 2]
y_train = y_train[y_train !=2]

print("Train data (n_samples, n_dimensions): {0}".format(X_train.shape))
classes, counts = np.unique(y_train, return_counts=True)
print("Classes {0}, samples per class {1}, {2}"
      .format(len(classes), counts[0], counts[1]))

Train data (n_samples, n_dimensions): (526, 175)
Classes 2, samples per class 313, 213


In [33]:
graph = nx.from_numpy_array(adjacency)
laplacian = nx.normalized_laplacian_matrix(graph)
vals, vecs = np.linalg.eigh(laplacian.toarray())
e = vals
U = vecs

X_f = (GFT(U, X_train.T)).T
X_f_heat = (graph_filter(U, X_train.T, heat_kernel, e, t=3)).T
X_f_rect = (graph_filter(U, X_train.T, rectangle_kernel, e, l_min=0.2, l_max=1)).T

X = [X_train, X_f, X_f_heat, X_f_rect]

**Do PCA**

In [34]:
for x in X:
    xtr_pca = do_pca(x, n_comps=50, plot_expl_var=False)
    xtr = x
    
    print(fit_log_regression(xtr_pca,y_train,20,k_fold=5))
    print(fit_log_regression(xtr,y_train,20,k_fold=5))
    

(526, 50)
(526, 175)
0.6501901140684411
0.7718631178707225
(526, 50)
(526, 175)
0.6653992395437263
0.7699619771863118
(526, 50)
(526, 175)
0.623574144486692
0.7414448669201521
(526, 50)
(526, 175)
0.5836501901140685
0.5950570342205324


**Do logistic regression**

In [ ]:
log_reg_accuracy = fit_log_regression(X,y_train,20,k_fold=5)

**Do SVM**

In [ ]:
c_hyperparam = np.logspace(-6,-1,11)
use_kernel = 'linear'
k_fold = 5

mean_accuracy, variance = fit_svm(X, y_train, use_kernel, c_hyperparam, k_fold=k_fold)

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(mean_accuracy, 'b.-')
ax1.set_ylabel('Accuracy', color='b')
ax2 = ax1.twinx()
ax2.plot(variance, 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

ConvNN:

In [ ]:
leg='50'
from_date = None
to_date = None
link_cutoff = 0.7
data_transf, adjacency, node_index, affairs_features, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-'+leg+'-0.csv',
                                                                     start_date=from_date, end_date=to_date,
                                                                     filter_method='number_NA',cutoff=10,ret_transf=True, delete_links_below=link_cutoff )

In [ ]:
hyperparam_values = np.logspace(-7,-2,15)
hyperparam_name = 'regularization'
k_fold = 5
nn_accuracy, nn_loss = cross_validate_convNN(X_train, y_train, adjacency, hyperparam_name, hyperparam_values, k_fold)

In [ ]:
nn_accuracy

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(hyperparam_values, np.array(nn_accuracy)[:,1], 'b.-')
ax1.set_ylabel('Accuracy', color='b')
ax1.set_xlabel(hyperparam_name)
ax2 = ax1.twinx()
ax2.plot(hyperparam_values, np.array(nn_loss)[:,1], 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

In [ ]:
hyperparam_values